# Diffie-Hellman Key Exchange

In [69]:
import hashlib
import random

import galois

# Define our field
GF = galois.GF(2**8)
G = GF.primitive_element


def rand_scalar():
    return random.randint(0, GF.order - 1)


# Alice generates a random scalar `a` and computes `A = g^a`.
a = rand_scalar()
A = G**a

# Bob generates a random scalar `b` and computes `B = g^b`.
b = rand_scalar()
B = G**b

# Alice and Bob exchange `A` and `B` over the insecure channel.

# Alice computes the shared secret key as `B^a`.
alice_S = B**a

# Bob computes the shared secret key as `A^b`.
bob_S = A**b

assert alice_S == bob_S

## XOR Encryption
We can use the shared secret key to encrypt and decrypt messages using the XOR cipher. Let's define a simple XOR encryption and decryption function.

In [70]:
def xor_encrypt(msg, key):
    return bytes([m ^ key[i % len(key)] for i, m in enumerate(msg)])


def xor_decrypt(ciphertext, key):
    return bytes([c ^ key[i % len(key)] for i, c in enumerate(ciphertext)])


# We transform the shared secret into a symmetric key by hashing it
shared_secret = int(alice_S)
symm_key = hashlib.sha256(shared_secret.to_bytes(length=shared_secret.bit_length(), byteorder="big")).digest()

# Alice encrypts a message to Bob using the shared secret key.
msg = b"Hello, Bob!"
ciphertext = xor_encrypt(msg, symm_key)

# Bob decrypts the message using the shared secret key.
decrypted_msg = xor_decrypt(ciphertext, symm_key)

assert decrypted_msg == msg